<a href="https://colab.research.google.com/github/duytran1511/Danh_gia_hieu_nang/blob/main/MM1-serial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installation of SimPy""
If not available**

In [14]:
# Window
!python -m pip install simpy
!python -m pip install numpy
!python -m pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Import of required modules**

In [15]:
import simpy
import numpy as np
import numpy.random as random

**Parameters settings**

In [16]:
MAXSIMTIME              = 50000
VERBOSE                 = False
LAMBDA                  = 3.8
MU                      = 8.0
POPULATION              = 50000000
SERVICE_DISCIPLINE      = 'SJF'
LOGGED                  = False
PLOTTED                 = False

**Discrete-Event Simulation model**


**The definition of a job**.

The properties of a job are


1. job execution time
2. job arrival time

In [17]:
class Job:
    def __init__(self, name, arrtime, duration):
        self.name = name
        self.arrtime = arrtime
        self.duration = duration

    def __str__(self):
        return '%s at %d, length %d' %(self.name, self.arrtime, self.duration)

**Disciplines**
Different queue disciplines can be defined here.


1.   Shortest Job First (SJF)
2.   List item



In [18]:
def SJF( job ):
    return job.duration

**The definition of server**

 There are 3 arguments needed for a server:
 1. env: SimPy environment
 2. queue discipline: 
   - FIFO: First In First Out
   - SJF : Shortest Job  
 3. name: name of sever 1 or 2

In [19]:
class Server:
    def __init__(self, env, strat = 'FIFO', name = 1):
        self.name = name
        
        self.env = env
        self.strat = strat
        self.Jobs = list(())
              
        self.serversleeping = None
        
        ''' statistics '''
        self.waitingTime = 0
        self.idleTime = 0
        self.jobsDone = 0

        ''' register a new server process '''
        env.process( self.serve() )

    def nJobs(self):
        return len(self.Jobs)

    def serve(self):
        while True:
            # do nothing, just change server to idle
            # and then yield a wait event which takes infinite time
            if len( self.Jobs ) == 0 :
                self.serversleeping = self.env.process( self.waiting( self.env ))
                t1 = self.env.now
                yield self.serversleeping
                # accumulate the server idle time
                self.idleTime += self.env.now - t1
            else:
                # get the first job to be served
                if self.strat == 'SJF':
                    self.Jobs.sort( key = SJF )
                    j = self.Jobs.pop( 0 )
                else: # FIFO by default
                    j = self.Jobs.pop( 0 )
                if LOGGED:
                    if self.name == 1:
                        qlog1.write( '%.4f\t%d\t%d\n' 
                            % (self.env.now, 1 if len(self.Jobs)>0 else 0, len(self.Jobs)) )
                    else:
                        qlog2.write( '%.4f\t%d\t%d\n' 
                            % (self.env.now, 1 if len(self.Jobs)>0 else 0, len(self.Jobs)) )

                ''' sum up the waiting time'''
                self.waitingTime += self.env.now - j.arrtime
                ''' yield an event for the job finish'''
                yield self.env.timeout( j.duration )



                ''' sum up the jobs done '''
                self.jobsDone += 1

    def waiting(self, env):
        try:
            if VERBOSE:
                print( 'Server', self.name, 'is idle at %.2f' % self.env.now )
            yield self.env.timeout( MAXSIMTIME )
        except simpy.Interrupt as i:
            if VERBOSE:
                print('Server', self.name ,'waken up and works at %.2f' % self.env.now )

**The arrival process**

The arrival process is exponentially distributed which is parameterized by
1. number of servers
2. maximum number of population
3. arrival rate $\lambda$
4. service rate $\mu$
*Note that, the implementation of the arrival process embeds both arrival and service distributions.*

In [20]:
class JobGenerator:
    def __init__(self, env, server1, server2, nrjobs = 10000000, lam = 5, mu = 8):
        self.server1 = server1
        self.server2 = server2

        self.nrjobs = nrjobs
        self.interarrivaltime = 1/lam;
        self.servicetime = 1/mu;
        env.process( self.generatejobs(env) )
        
    def generatejobs(self, env):
        i = 1
        while True:
            # yield an event for new job arrival
            job_interarrival = random.exponential( self.interarrivaltime )
            yield env.timeout( job_interarrival )

            # generate service time and add job to the list
            job_duration = random.exponential( self.servicetime )

            server = 1
            # If the number of jobs in both queues is equal, random 50%
            if self.server1.nJobs() == self.server2.nJobs():
                temp = random.randint(1, 100)

                if temp <= 50:
                    server = 1
                else:
                    server = 2
            # If n jobs of server 1 is fewer, choose server 1
            elif self.server1.nJobs() < self.server2.nJobs():
                server = 1
            # If n jobs of server 2 is fewer, choose server 2
            else:
                server = 2

            if server == 1:
                self.server1.Jobs.append( Job('Job %s' %i, env.now, job_duration) )
            else:
                self.server2.Jobs.append( Job('Job %s' %i, env.now, job_duration) )

            if VERBOSE:
                print( 'job %d: t = %.2f, l = %.2f, dt = %.2f' 
                    %( i, env.now, job_duration, job_interarrival ) )

            i += 1

            if server == 1:
                ''' if server 1 is idle, wake it up'''
                if not self.server1.serversleeping.triggered:
                    self.server1.serversleeping.interrupt( 'Wake up server 1, please.' )
            else:
                ''' if server 2 is idle, wake it up'''
                if not self.server2.serversleeping.triggered:
                    self.server2.serversleeping.interrupt( 'Wake up server 2, please.' )

**Open the log file**

If requested.

In [21]:
if LOGGED:
    qlog1 = open( 'mm1-l%d-m%d1.csv' % (LAMBDA,MU), 'w' )
    qlog1.write( '0\t0\t0\n' )
    qlog2 = open( 'mm1-l%d-m%d2.csv' % (LAMBDA,MU), 'w' )
    qlog2.write( '0\t0\t0\n' )

**Start SimPy environment**

In [22]:
env = simpy.Environment()

MyServer1 = Server( env, SERVICE_DISCIPLINE, 1 )
MyServer2 = Server( env, SERVICE_DISCIPLINE, 2 )

MyJobGenerator = JobGenerator( env, MyServer1, MyServer2, POPULATION, LAMBDA, MU )

**Run the simulation** 

In [23]:
env.run( until = MAXSIMTIME )

**Close the log file**

In [24]:
if LOGGED:
    qlog1.close()
    qlog2.close()

**Print some statistics**

In [25]:
RHO = LAMBDA/MU

RHO1 = (LAMBDA / 2) / MU
RHO2 = (LAMBDA / 2) / MU


print('Arrivals 1               : %d' % (MyServer1.jobsDone) )
print('Arrivals 2               : %d\n' % (MyServer2.jobsDone) )


print('Idle time 1              : %d' % (MyServer1.idleTime))
print('Idle time 2              : %d\n' % (MyServer2.idleTime))


print('Waiting time 1           : %d' % (MyServer1.waitingTime))
print('Waiting time 2           : %d\n' % (MyServer2.waitingTime))


print('Job done 1               : %d' % (MyServer1.jobsDone))
print('Job done 2               : %d\n' % (MyServer2.jobsDone))


print('Utilization 1            : %.2f/%.2f' 
    % (1.0-MyServer1.idleTime/MAXSIMTIME, RHO1) )
print('Utilization 2            : %.2f/%.2f\n' 
    % (1.0-MyServer2.idleTime/MAXSIMTIME, RHO2) )
    

print('Mean waiting time 1      : %.2f/%.2f' 
    % (MyServer1.waitingTime/MyServer1.jobsDone, RHO1**2/((1-RHO1)*LAMBDA) ) )

print('Mean waiting time 2      : %.2f/%.2f' 
    % (MyServer2.waitingTime/MyServer2.jobsDone, RHO2**2/((1-RHO2)*LAMBDA) ) )

Arrivals 1               : 95767
Arrivals 2               : 94383

Idle time 1              : 38022
Idle time 2              : 38200

Waiting time 1           : 2558
Waiting time 2           : 2464

Job done 1               : 95767
Job done 2               : 94383

Utilization 1            : 0.24/0.24
Utilization 2            : 0.24/0.24

Mean waiting time 1      : 0.03/0.02
Mean waiting time 2      : 0.03/0.02


**Plot the statistics**

In [26]:
if LOGGED and PLOTTED:
    import matplotlib.pyplot as plt
    log = np.loadtxt( 'mm1-l%d-m%d1.csv' % (LAMBDA,MU), delimiter = '\t' )
    plt.subplot( 2, 1, 1 )
    plt.xlabel( 'Time' )
    plt.ylabel( 'Queue length' )
    plt.step( log[:200,0], log[:200,2], where='post' )
    plt.subplot( 2, 1, 2 )
    plt.xlabel( 'Time' )
    plt.ylabel( 'Server state' )
    plt.yticks([0, 1], ['idle', 'busy'])
    #plt.step( log[:200,0], log[:200,1], where='post' )
    plt.fill_between( log[:200,0], 0, log[:200,1], step="post", alpha=.4 )
    plt.tight_layout()
    plt.show()

if LOGGED and PLOTTED:
    import matplotlib.pyplot as plt
    log = np.loadtxt( 'mm1-l%d-m%d2.csv' % (LAMBDA,MU), delimiter = '\t' )
    plt.subplot( 2, 1, 1 )
    plt.xlabel( 'Time' )
    plt.ylabel( 'Queue length' )
    plt.step( log[:200,0], log[:200,2], where='post' )
    plt.subplot( 2, 1, 2 )
    plt.xlabel( 'Time' )
    plt.ylabel( 'Server state' )
    plt.yticks([0, 1], ['idle', 'busy'])
    #plt.step( log[:200,0], log[:200,1], where='post' )
    plt.fill_between( log[:200,0], 0, log[:200,1], step="post", alpha=.4 )
    plt.tight_layout()
    plt.show()